# Import Important Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import splitfolders
from tensorflow import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from keras.layers import *
from keras.preprocessing import *
from keras.preprocessing.image import *
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import random as random
import tensorflow_addons as tfa
import matplotlib as mpl
import warnings
import os
import cv2

# Ignore warnings

In [ ]:
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Set Random Values

In [ ]:
seed_value=40
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Count Number Of Images

In [ ]:
ImagePath = './Kideny'

In [ ]:
from pathlib import Path

Data_Root = Path(ImagePath)

Total = 0
for sub_dir in Data_Root.iterdir():
    Count = len(list(sub_dir.iterdir()))
    Total += Count
    print(f'{sub_dir.name}: {Count}')

print('')
print(f'Total Numbers Of Files: {Total}')

# Split The Data Into Train, Test, And Validation

In [ ]:
directory = "Data" 
parent_dir = '/Users/osamaanmar/Downloads/Kideny Disease Classification/'
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [ ]:
Input = ImagePath
Output = './Data'

splitfolders.ratio(Input, output=Output, seed=seed_value, ratio=(.8, 0.1, .1))

# Generate Image DataSet

In [ ]:
Training_Set = tf.keras.preprocessing.image_dataset_from_directory(directory='/Users/osamaanmar/Downloads/Kideny Disease Classification/Data/train',
                                               label_mode='categorical',
                                               seed=seed_value,
                                               image_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

In [ ]:
Testing_Set = tf.keras.preprocessing.image_dataset_from_directory(directory='/Users/osamaanmar/Downloads/Kideny Disease Classification/Data/test',
                                               label_mode='categorical',
                                               seed=seed_value,
                                               image_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

In [ ]:
Validation_Set = tf.keras.preprocessing.image_dataset_from_directory(directory='/Users/osamaanmar/Downloads/Kideny Disease Classification/Data/val',
                                               label_mode='categorical',
                                               seed=seed_value,
                                               image_size=(160, 160),
                                               batch_size=32,
                                               shuffle=True)

# Data Visualization 

In [ ]:
def plot_some_images(Data, NumberOfImages):
    Labels = Data.class_names
    Data = Data
    NumberOfImages = NumberOfImages

    Row  = int(tf.get_static_value(math.ceil(NumberOfImages**0.5)))
    Col  = int(tf.get_static_value(math.ceil(NumberOfImages**0.5)))

    Counter = 1

    plt.figure(figsize=(15, 15))
    for images, labels in Data:
     for i in range(NumberOfImages):
        ax = plt.subplot(Row, Col,Counter)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(Labels[np.argmax(labels[i])])
        plt.axis("off")

        if Counter >+ NumberOfImages:
           break
        else:
           Counter += 1

**1. Training Set**

In [ ]:
plot_some_images(Training_Set, 20)

**2.Testing Set**

In [ ]:
plot_some_images(Testing_Set, 20)

**3. Validation Set**

In [ ]:
plot_some_images(Validation_Set, 20)

# Models

In [ ]:
Input_Shape = (160, 160, 3)
Epochs = 100
Number_Of_Classes = len(os.listdir(ImagePath))

In [ ]:
Metrics = ['accuracy',
           tf.keras.metrics.Precision(name='Precision'), 
           tfa.metrics.F1Score(name='F1', num_classes=Number_Of_Classes, average='micro'),
           tf.keras.metrics.Recall(name='Recall'),]

In [ ]:
Callback = tf.keras.callbacks.EarlyStopping(
                    monitor="accuracy",
                    patience=5,
                    verbose=0,
                    mode="auto",
                    restore_best_weights=True,
                    start_from_epoch=10,
                )

In [ ]:
class Models(): 
    def __init__(self,input_shape,epochs) :
        self.input_shape = input_shape
        self.epochs = epochs
        
    def convolutional_neural_network(self):

        model = keras.models.Sequential([
                                         Conv2D(32, 3, activation='relu', input_shape = self.input_shape),
                                         keras.layers.MaxPooling2D(),
                                         
                                         keras.layers.Conv2D(32, 3, activation='relu'),
                                         keras.layers.MaxPooling2D(),

                                         keras.layers.Conv2D(64, 3, activation='relu'),
                                         keras.layers.MaxPooling2D(),
                                        
                                         keras.layers.Flatten(),
                                         Dropout(0.2),

                                         keras.layers.Dense(128 ,activation='relu'),
                                        
                                         keras.layers.Dense(Number_Of_Classes, activation='softmax'),

                                         ])
            
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='ADAM',
                        metrics=Metrics)
        
        model.fit(Training_Set, 
                  epochs= self.epochs, 
                  validation_data=Validation_Set,
                  callbacks=[Callback],
                  shuffle=True
                    );
        
        return model
    
    
    def tranfer_learning(self, base_model):  

        base_model = base_model
        base_model.trainable = False
            
        inputs = tf.keras.Input(self.input_shape)
        x = base_model(inputs)
        x = GlobalAveragePooling2D()(x)
        x = Flatten()(x)
        x = Dropout(0.2)(x)
        outputs = Dense(Number_Of_Classes, 'sigmoid')(x)
        model = tf.keras.Model(inputs, outputs)
            
            
        model.compile(loss= 'categorical_crossentropy',
                        optimizer='Adam',
                        metrics=Metrics)

        model.fit(Training_Set,
                  epochs=self.epochs,
                  validation_data=Testing_Set,
                  callbacks=[Callback], 
                  shuffle=True,
                  );
        
        return model    

# Model History Result

In [ ]:
def model_history_values(model, name):
    file_name = name
    HistoryResult= pd.DataFrame(model.history.history)
    HistoryResult.to_csv(os.path.join('{}HitoryResult.csv'.format(file_name)) ,index=False)
    return HistoryResult

# Model Evaluation

In [ ]:
def model_evulation(model, data):
    Loss, Accuracy, Precision, F1Score, Recall = model.evaluate(data, verbose=0)
    
    return Loss, Accuracy, Precision, F1Score, Recall   

# Generate Real And Prediction Classes

In [ ]:
def generate_y(model, test_dir):
    test_dir = test_dir
    Real = []
    for label in os.listdir(test_dir):
        for image in os.listdir(test_dir+label):
            Real.append(label)

    encoder =  LabelEncoder()
    Real = encoder.fit_transform(Real)
            
    Prediction = model.predict(Testing_Set)
    Prediction = np.argmax(Prediction, axis=1)
    
    return Real, Prediction


# Confusion Matrix

In [ ]:
def confusion_matrix(Real, Prediction):
        
        ClassesName = Testing_Set.class_names
        ConfusionMatrix = tf.math.confusion_matrix(Real,Prediction).numpy()
        ConfusionMatrix = pd.DataFrame(ConfusionMatrix,
                     index = ClassesName, 
                     columns = ClassesName)
        return ConfusionMatrix
    
def plot_confusion_matrix(ConfusionMatrix, name):

        mpl.style.use('seaborn')
        plt.figure(figsize= (10, 10))
        sns.heatmap(ConfusionMatrix, square=True,annot=True, cmap="BrBG", fmt="d", linewidths=.5)
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.title('{} Confusion Matrix with Labels'.format(name))
        plt.rcParams['font.size'] = '10'
        plt.grid(None)
        plt.show();

# Convolutional Neural Network

In [ ]:
test_dir = '/Users/osamaanmar/Downloads/Kideny Disease Classification/Data/test/'

In [ ]:
name = 'Convolutional Neural Network'

In [ ]:
model = Models(Input_Shape, Epochs)
model0 = model.convolutional_neural_network()

In [ ]:
ModelHistory0 = model_history_values(model0, name)

In [ ]:
ModelHistory0

In [ ]:
TestLoss0, TestAccuracy0, TestPrecision0, TestF1Score0, TestRecall0 = model_evulation(model0,Testing_Set)

In [ ]:
Real0, Prediction0 = generate_y(model0, test_dir)

In [ ]:
ConfusionMatrix0 = confusion_matrix(Real0, Prediction0)

In [ ]:
plot_confusion_matrix(ConfusionMatrix0, name)

# Transfer Learning

**1. EfficientNetV2B0**

In [ ]:
name = 'EfficientNetV2B0'

In [ ]:
base_model = tf.keras.applications.EfficientNetV2B0(
                                                    include_top=False,
                                                    weights='imagenet')

NetV2 = Models(Input_Shape, Epochs)
model1 = NetV2.tranfer_learning(base_model)

In [ ]:
ModelHistory1 = model_history_values(model1, name)

In [ ]:
ModelHistory1

In [ ]:
TestLoss1, TestAccuracy1, TestPrecision1, TestF1Score1, TestRecall1 = model_evulation(model1,Testing_Set)

In [ ]:
Real1, Prediction0 = generate_y(model1, test_dir)

In [ ]:
ConfusionMatrix0 = confusion_matrix(Real0, Prediction0)

In [ ]:
plot_confusion_matrix(ConfusionMatrix0, name)

**2. ResNet101**

In [ ]:
name = 'Convolutional Neural Network'

In [ ]:
base_model = tf.keras.applications.ResNet101(
                                                    include_top=False,
                                                    weights='imagenet')

Res = Models(Input_Shape, Epochs)
model2 = Res.tranfer_learning(base_model)

In [ ]:
ModelHistory2 = model_history_values(model2, name)

In [ ]:
ModelHistory2

In [ ]:
TestLoss2, TestAccuracy2, TestPrecision2, TestF1Score2, TestRecall2 = model_evulation(model2,Testing_Set)

In [ ]:
Real2, Prediction2 = generate_y(model2, test_dir)

In [ ]:
ConfusionMatrix2 = confusion_matrix(Real2, Prediction2)

In [ ]:
plot_confusion_matrix(ConfusionMatrix2, name)

**3. InceptionV3**

In [ ]:
name = 'InceptionV3'

In [ ]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
        include_top=False,
        weights='imagenet',)

Inc = Models(Input_Shape, Epochs)
model3 = Inc.tranfer_learning(base_model)

In [ ]:
ModelHistory3 = model_history_values(model3, name)

In [ ]:
ModelHistory3

In [ ]:
TestLoss3, TestAccuracy3, TestPrecision3, TestF1Score3, TestRecall3 = model_evulation(model3, Testing_Set)

In [ ]:
Real3, Prediction3 = generate_y(model3, test_dir)

In [ ]:
ConfusionMatrix3 = confusion_matrix(Real3, Prediction3)

In [ ]:
plot_confusion_matrix(ConfusionMatrix3, name)

**4. VGG16**

In [ ]:
name = 'VGG16'

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',)

VGG = Models(Input_Shape, Epochs)
model4 = VGG.tranfer_learning(base_model)

In [ ]:
ModelHistory4 = model_history_values(model4, name)

In [ ]:
ModelHistory4 

In [ ]:
TestLoss4, TestAccuracy4, TestPrecision4, TestF1Score4, TestRecall4 = model_evulation(model4, Testing_Set)

In [ ]:
Real4, Prediction4 = generate_y(model4, test_dir)

In [ ]:
ConfusionMatrix4 = confusion_matrix(Real4, Prediction4)

In [ ]:
plot_confusion_matrix(ConfusionMatrix4, name)

**5. VGG19**

In [ ]:
name = 'VGG19'

In [ ]:
base_model = tf.keras.applications.vgg19.VGG19(
        include_top=False,
        weights='imagenet',)

VGG19 = Models(Input_Shape, Epochs)
model5 = VGG19.tranfer_learning(base_model)

In [ ]:
ModelHistory5 = model_history_values(model5, name)

In [ ]:
ModelHistory5

In [ ]:
TestLoss5, TestAccuracy5, TestPrecision5, TestF1Score5, TestRecall5 = model_evulation(model5, Testing_Set)

In [ ]:
Real5, Prediction5 = generate_y(model5, test_dir)

In [ ]:
ConfusionMatrix5 = confusion_matrix(Real5, Prediction5)

In [ ]:
plot_confusion_matrix(ConfusionMatrix5, name)

**6. Xception**

In [ ]:
name = 'Xception'

In [ ]:
base_model = tf.keras.applications.xception.Xception(
        include_top=False,
        weights='imagenet',)

Xce = Models(Input_Shape, Epochs)
model6 = Xce.tranfer_learning(base_model)

In [ ]:
ModelHistory6 = model_history_values(model6, name)

In [ ]:
TestLoss6, TestAccuracy6, TestPrecision6, TestF1Score6, TestRecall6 = model_evulation(model6, Testing_Set)

In [ ]:
Real6, Prediction6 = generate_y(model6, test_dir)

In [ ]:
ConfusionMatrix6 = confusion_matrix(Real6, Prediction6)

In [ ]:
plot_confusion_matrix(ConfusionMatrix6, name)

**7. EfficientNetB0**

In [ ]:
name = 'EfficientNetB0'

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(
        include_top=False,
        weights='imagenet',)

EFF = Models(Input_Shape, Epochs)
model7 = EFF.tranfer_learning(base_model)

In [ ]:
ModelHistory7 = model_history_values(model1, name)

In [ ]:
ModelHistory7

In [ ]:
TestLoss7, TestAccuracy7, TestPrecision7, TestF1Score7, TestRecall7 = model_evulation(model7, Testing_Set)

In [ ]:
Real7, Prediction7 = generate_y(model7, test_dir)

In [ ]:
ConfusionMatrix7 = confusion_matrix(Real7, Prediction7)

In [ ]:
plot_confusion_matrix(ConfusionMatrix7, name)